In [258]:
import pandas as pd
df = pd.read_csv("owid-covid-data.csv", encoding="latin1", low_memory=False)

#importo il dataset

In [259]:
# innanzitutto verifico le dimensioni del dataset ed i campi che lo compongono per capire come andare ad operare su di esso
df.shape

(311365, 67)

In [260]:
# valuto il formato delle colonne
df.dtypes

iso_code                                    object
continent                                   object
location                                    object
date                                        object
total_cases                                float64
                                            ...   
population                                 float64
excess_mortality_cumulative_absolute       float64
excess_mortality_cumulative                float64
excess_mortality                           float64
excess_mortality_cumulative_per_million    float64
Length: 67, dtype: object

In [261]:
#esploro il dataset
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [386]:
"""
    dopo una prima verifica del dataset ho identificato le colonne utili alla mia analisi, decido quindi di creare un nuovo dataset
    più leggero con all'interno solo i dati in oggetto nel report, per i calcoli relativi hai casi covid ho deciso di utilizzare la colonna
    new_cases dove vengono riportati tutti i nuovi casi giorno per giorno, mentre per le vaccinazioni non essendo aggiornati giorno per giorno
    questi dati prenderò come riferimento il valore massimo più aggiornato nella colonna total_vaccinations
"""
col = ["continent","location","new_cases","total_vaccinations","population","total_cases"]
dataset = df.loc[:,col]

In [387]:
# raggruppo per continente in un nuovo dataset calcolando inoltre la somma dei casi giornalieri
subset1 = dataset.groupby(["continent","location"]).agg({"new_cases":"sum","population":"max"})
subset1.rename(columns={"new_cases":"total_cases"},inplace=True)

# vado ad esporre solo le colonne di mio interesse
subset1.loc[:,["total_cases","population"]].astype("int64")

total_cases  population
continent     location                             
Africa        Algeria            271817    44903228
              Angola             105384    35588996
              Benin               28014    13352864
              Botswana           329862     2630300
              Burkina Faso        22056    22673764
...                                 ...         ...
South America Paraguay           735759     6780745
              Peru              4505220    34049588
              Suriname            82505      618046
              Uruguay           1037922     3422796
              Venezuela          552695    28301700

[243 rows x 2 columns]

In [388]:
"""
    prima vado a raggruppare per continente su successivamente aggiungo un colonna con la 
    percentuale di casi in relazione alla popolazione
"""
subset = subset1.groupby("continent").agg({"total_cases":"sum","population":"sum"})
subset.insert(1, "perc. pop.", subset["total_cases"].values*100/subset["population"])
subset[["total_cases","population"]] = subset[["total_cases","population"]].astype("int64")
subset

,total_cases,perc. pop.,population
continent,,,
Africa,13088991,0.917408,1426736614
Asia,297234406,6.294888,4721838226
Europe,249319188,30.610344,814493270
North America,124175560,20.684769,600323657
Oceania,14073975,31.248483,45038907
South America,68568119,15.697230,436816679


In [269]:
"""
    i 2 continenti da me presi in oggetto saranno ASIA e OCEANIA, vado ora a creare un ulteriore dataset che prenda 
    in esame solo i dati relativi a tali ontinenti
"""
cond1 = dataset.continent == "Asia"
cond2 = dataset.continent == "Oceania"
subset2 = dataset.loc[cond1 | cond2, ["location","continent","new_cases"]]

In [369]:
# ora aggrego i dati per nazione e calcolo la somma dei casi totali
subset3 = subset2.groupby(["continent","location"]).sum("new_cases")
subset3.rename(columns={"new_cases":"total_cases"},inplace=True)
subset3.astype("int64")

total_cases
continent location                      
Asia      Afghanistan             218720
          Armenia                 449148
          Azerbaijan              831659
          Bahrain                 696614
          Bangladesh             2038539
...                                  ...
Oceania   Tokelau                      5
          Tonga                    16817
          Tuvalu                    2779
          Vanuatu                  12016
          Wallis and Futuna         3427

[75 rows x 1 columns]

In [382]:
"""
    attraverso le operazioni precedenti ho potuto notare la mancanza di alcuni dati in alcune nazioni, questo sembra impattare negativamente
    sulle operazioni matematiche del report, andrò quindi ad eliminarli considerandoli dati errati/mancanti
"""
import numpy as np
subset3 = subset3.replace(0, np.nan)

In [378]:
"""
    ora sto creando un set per visualizzare i valori minimi di casi dei continenti presi in esame 
    confrontandoli con il numero di casi totali mondiali
"""
dmin = pd.DataFrame({"Asia":subset3.loc["Asia"].min(skipna=True),"Oceania":subset3.loc["Oceania"].min(),"Mondo":dataset["total_cases"].max()})
dmin.astype("int64")

,Asia,Oceania,Mondo
total_cases,11945,4,766440032


In [379]:
"""
    faccio la stessa cosa sia per il massimo che per la media casi, come per la richesta precedente usero come valore di riferimento il dato
    massimo estratto dalla colonna total_cases del dataset iniziale
"""
dmax = pd.DataFrame({"Asia":subset3.loc["Asia"].max(),"Oceania":subset3.loc["Oceania"].max(),"Mondo":dataset["total_cases"].max()})
dmax.astype("int64")

,Asia,Oceania,Mondo
total_cases,99261436,11299954,766440032


In [380]:
# alcuni valori durante l'analisi verranno trasfromati in interi per renderli più leggibili
dmed = pd.DataFrame({"Asia":int(subset3.loc["Asia"].mean()),"Oceania":subset3.loc["Oceania"].mean(),"Mondo":dataset["total_cases"].max()})
dmed.astype("int64")

,Asia,Oceania,Mondo
total_cases,6605209,586415,766440032


In [381]:
# ora calcolo la percentuale dei continenti sul numero totale di casi mondiale
dper = pd.DataFrame({"Asia":(subset3.loc["Asia"].sum()*100)/dataset["total_cases"].max(),"Oceania":(subset3.loc["Oceania"].sum()*100)/dataset["total_cases"].max()})
dper.rename(index={"total_cases":"total_cases %"},inplace=True)
dper

,Asia,Oceania
total_cases %,38.781169,1.836279


In [305]:
# vado ora creare un nuovo set che mi permetta di svolgere le operazioni riguardanti i dati sulle vaccinazioni
subset4 = dataset.loc[:,["continent","location","total_vaccinations"]]
subset5 = subset4.groupby(["continent","location"]).max("total_vaccinations")
subset5


total_vaccinations
continent     location                        
Africa        Algeria               15267442.0
              Angola                25402678.0
              Benin                  4232541.0
              Botswana               3171667.0
              Burkina Faso           6674010.0
...                                        ...
South America Paraguay               9745545.0
              Peru                  89421273.0
              Suriname                505699.0
              Uruguay                9021046.0
              Venezuela             37860994.0

[243 rows x 1 columns]

In [426]:
""" 
    diversamente da prima la mancanza di dati di alcune nazioni non modifica il risultato delle analisi quindi non andrò ad eliminare
    tali valori, ora però devo raggruppare i valori per continente come richiesto dalla traccia
"""
subset6 = subset5.groupby("continent").sum()
subset6.astype("int64")

,total_vaccinations
continent,
Africa,805392236
Asia,9006979649
Europe,1552519886
North America,1149963868
Oceania,81007629
South America,957020867


In [421]:
""" 
    diversamente da prima la mancanza di dati di alcune nazioni non modifica il risultato delle analisi quindi non andrò ad eliminare
    tali valori, ora però devo raggruppare i valori per continente come richiesto dalla traccia
"""
vacc = pd.DataFrame({"Africa":subset5.loc["Africa"].sum(),
                     "Asia":subset5.loc["Asia"].sum(),
                     "Europe":subset5.loc["Europe"].sum(),
                     "North America":subset5.loc["North America"].sum(),
                     "Oceania":subset5.loc["Oceania"].sum(),
                     "South America":subset5.loc["South America"].sum(),
                     "Mondo":dataset["total_vaccinations"].max()})
vacc.astype("int64")

,Africa,Asia,Europe,North America,Oceania,South America,Mondo
total_vaccinations,805392236,9006979649,1552519886,1149963868,81007629,957020867,13381892700


In [294]:
vmin = pd.DataFrame({"Africa":subset5.loc["Africa"].min(),
                     "Asia":subset5.loc["Asia"].min(),
                     "Europe":subset5.loc["Europe"].min(),
                     "North America":subset5.loc["North America"].min(),
                     "Oceania":subset5.loc["Oceania"].min(),
                     "South America":subset5.loc["South America"].min(),
                     "Mondo":dataset["total_vaccinations"].max()})
vmin.astype("int64")

,Africa,Asia,Europe,North America,Oceania,South America,Mondo
total_vaccinations,7892,861519,65140,4619,117,4407,13381892700


In [295]:
vmax = pd.DataFrame({"Africa":subset5.loc["Africa"].max(),
                     "Asia":int(subset5.loc["Asia"].max()),
                     "Europe":subset5.loc["Europe"].max(),
                     "North America":subset5.loc["North America"].max(),
                     "Oceania":subset5.loc["Oceania"].max(),
                     "South America":subset5.loc["South America"].max(),
                     "Mondo":dataset["total_vaccinations"].max()})
vmax.astype("int64")

,Africa,Asia,Europe,North America,Oceania,South America,Mondo
total_vaccinations,116606863,3491077000,192221468,676728782,63681652,486436436,13381892700


In [297]:
vmed = pd.DataFrame({"Africa":subset5.loc["Africa"].mean(),
                     "Asia":subset5.loc["Asia"].mean(),
                     "Europe":subset5.loc["Europe"].mean(),
                     "North America":subset5.loc["North America"].mean(),
                     "Oceania":subset5.loc["Oceania"].mean(),
                     "South America":subset5.loc["South America"].mean(),
                     "Mondo":dataset["total_vaccinations"].max()})
vmed.astype("int64")

,Africa,Asia,Europe,North America,Oceania,South America,Mondo
total_vaccinations,14914671,180139592,28750368,33822466,4500423,73616989,13381892700


In [427]:
# creo un dizionario con i valori percentuali per poterlo utilizzare successivamente
vper = pd.DataFrame({"Africa":(subset5.loc["Africa"].sum()*100)/dataset["total_vaccinations"].max(),
                     "Asia":(subset5.loc["Asia"].sum()*100)/dataset["total_vaccinations"].max(),
                     "Europe":(subset5.loc["Europe"].sum()*100)/dataset["total_vaccinations"].max(),
                     "North America":(subset5.loc["North America"].sum()*100)/dataset["total_vaccinations"].max(),
                     "Oceania":(subset5.loc["Oceania"].sum()*100)/dataset["total_vaccinations"].max(),
                     "South America":(subset5.loc["South America"].sum()*100)/dataset["total_vaccinations"].max()})
vper.rename(index={"total_vaccinations":"total_vaccinations %"},inplace=True)

In [434]:
# ora vado a creare un report finale con la somma ed il valore percentuale per il report finale
subset7 = vper.T
subset7.rename_axis("continent", inplace=True)
subset8 = pd.merge(left=subset6, right=subset7, how="inner", on="continent")
subset8["total_vaccinations"] = subset8["total_vaccinations"].astype("int64")
subset8

,total_vaccinations,total_vaccinations %
continent,,
Africa,805392236,6.018523
Asia,9006979649,67.307218
Europe,1552519886,11.601646
North America,1149963868,8.593432
Oceania,81007629,0.605353
South America,957020867,7.151611


In [442]:
"""
    mettendo in relazione in casi covid totali si evince che i continenti maggiormente colpiti in sono
    asia, europa e nord america tuttavia confrontando il dato relativo calcolato in base alla popolazione
    si può notare come l'oceania abbia il dato più elevato in percentuale di popolazione colpita, invece
    l'asia nonstante domini la classifica per numero di casi abbia una percentuale relativamente bassa 
    di popolazione contagiata.
    mettendo in relazione proprio su questi ultimi due continenti per la mia seconda analisi si può notare
    come, nonstante l'asia abbia una popolazione numericamente maggiore, i due continenti siano simili in questo caso
    perchè entrambi hanno massimo molto elevato ma anche un valore minino molto basso, questo indica che in entrambi i 
    continenti ci sono sia nazioni molto grandi sia molto piccole. il dato relativo alla percentuale sui casi totali mondiali
    invece è completamente a favore dell'asia in quanto è il continente con il valore assoluto maggiore.
    nell'ultima analisi si può constatare che nei continenti dove ci sono stati il maggior numero di casi vi sono state
    anche maggiormente le vaccinazioni. in nord america nonstante un elevato numero di casi ed un maggior numero di popolazione 
    c'è un numero simile di vaccinazioni, sorprende l'oceania che nonostante un numero in percentuale elevato di contagi abbia 
    un basso numero di vaccinati.
"""
    
    